In [7]:
import genjax
import jax
from genjax import nablasp

console = genjax.pretty()

In [8]:
nablasp

<module 'genjax.nablasp' from '/home/femtomc/Research/genjax/src/genjax/nablasp.py'>

In [9]:
@genjax.gen
def model():
    x = dippl.normal_reparam(0.0, 10.0) @ "x"
    y = dippl.normal_reparam(0.0, 10.0) @ "y"
    rs = x**2 + y**2
    z = dippl.normal_reparam(rs, rs / 10.0) @ "z"

In [10]:
# Now, we define our variational proposal.
@genjax.gen
def variational_family(ϕ):
    μ1, μ2, log_σ1, log_σ2 = ϕ
    x = dippl.normal_reparam(μ1, jnp.exp(log_σ1)) @ "x"
    y = dippl.normal_reparam(μ2, jnp.exp(log_σ2)) @ "y"

In [11]:
loss = nablasp.elbo(model, (), variational_family, (), genjax.choice_map({"z": 3.0}))

In [12]:
loss



Expectation
└── prog
    └── ADEVProgram
        └── source
            └── <wrapped function elbo_loss>